# SPARK GROUP C ASSIGNMENT

**Purpose:** Data Analysis to find out the <font color=blue >**EATING HABITS IN EUROPEAN COUNTRIES**</font> <br>
**Focus Country:** <font color=blue >Germany</font>

**Author:** S2-3 (Group C) <br>
**Contact:** <br> pierremulliez@student.ie.edu <br> mate.vilic@student.ie.edu <br> elsaarnaiz@student.ie.edu <br> olga.frech@student.ie.edu <br> alienor@student.ie.edu <br> ggerman.souza@student.ie.edu <br> andreimenshchikov@student.ie.edu <br> jorge.campos@student.ie.edu

**Client:** Prof. Raúl Marín & <font color=red >ACME CORPORATION</font>  

**Code created:** 2021-02-21 <br>
**Last updated:** 2021-03-07

**Comment:** It is all about Food Products 


## Agenda
1. PySpark **environment setup**
2. Data source and **Spark data abstraction** - DataFrame **set up**
3. Exploratory **data analysis**
4. **Subset** data analysis
    1. Find the **oldest product**
    2. Find the **newest product**
    3. **Average product age**, where age means how long the product has been in the system
    4. **List of other countries** where products are sold too
    5. Identify **category of products** and the compute:
  1. **Number** of **products** by category
  2. **List** containing names of **products by category** 
    6. Identify traces and compute:
  1. **Number** of products **by trace**
  2. **List** containing names of **products by trace**
    7. Data quality analysis on **fields of interest** (see appendix 1):
  1. Number of products with **complete info**
  2. % of products **without complete analysis per 100g**
  3. % of products **without additives info**
  4. % of products **without traces info**
    8. **Data profiling** on fields of interest (see appendix 1):
  1. Stats on **analysis per 100g fields**
5. Analysis to find out **which products are safe**

Let's start:

## 1. PySpark environment setup

In [1]:
#Importing APIs

import pandas as pd
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType
from pyspark.sql.window import Window
from IPython.display import display, Markdown
from pyspark.sql import SQLContext
from pyspark.sql import functions as F

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

sqlContext = SQLContext(spark)

## 2. Data source and Spark data abstraction - DataFrame setup

In [3]:
#Load dataset

resto = spark.read \
                 .option("inferSchema", "true") \
                 .option("delimiter", "\t") \
                 .option("mode","PERMISSIVE") \
                 .option("header", "false") \
                 .csv("noTT.csv")
concate = spark.read \
                 .option("inferSchema", "true") \
                 .option("delimiter", "\t") \
                 .option("mode","PERMISSIVE") \
                 .option("header", "true") \
                 .csv("complete.csv")
tt = spark.read \
                 .option("inferSchema", "true") \
                 .option("delimiter", ";") \
                 .option("mode","PERMISSIVE") \
                 .option("header", "true") \
                 .csv("title.csv")

In [4]:
resto.show(3)
rest = resto.drop("C_185")
#.drop("C_173").drop("C_174").drop("C_175")
rest.show(1)
tt.show(1)
concate_with_title = tt.union(rest)
 

+----------+--------------------+-------------+----------+-------------------+----------+-------------------+--------------------+----+----------------+-----+---------+---------+----+------------------+------------------+--------------------+--------------------+--------------------+----+----+----+-------+-------+--------------------+--------------------+--------------------+----+----+----+----+----+----+----+-----------+----------+-------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+-----+----+----+----+--------------------+--------------------+----+----+----+----+----+----+----+----+----+-------------+-------+--------------------+--------------------+--------------------+----+--------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------+-----+------+----+----+---------------+----+----+----+----+----+----+----+----+--

## 3. Exploratory **data analysis**

In [5]:
concate.show(2)

+----+--------------------+-------------+----------+-------------------+---------------+----------------------+--------------------+------------------------+------------+--------+---------+--------------+--------------+------------------+------------------+-----------+---------------+-------------+-------+------------+----------+--------------------+-------------------------+-------------+----------------+-------------+---------+--------------+------------------------+------+-----------+---------------+------+---------+--------------+------------+----------------+---------+------------+------+-----------+---------+------------+----------------+-------------+-----------+---------+--------------+------------+---------------------------+-------------------------+------------------------------+---------------------------------------+-------------------------------------+------------------------------------------+----------------+----------------+----------+-------------+-------------+-----

In [6]:
pdcont = concate.toPandas()

In [7]:
pd.set_option('display.max_columns', None)
pdcont.head(5)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,brand_owner,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,17.0,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1591989744,2020-06-12 21:22:24,1609478763,2021-01-01 06:26:03,Vitória crackers,None,None,None,barquette,barquette,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Allemagne,en:germany,Germany,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,NaN,None,None,NaN,None,None,NaN,None,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",None,None,None,None,None,None,None,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,375.0,1569.0,NaN,7.0,3.08,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,70.099998,1

In [8]:
pdcont.describe()

,code,created_t,last_modified_t,serving_quantity,additives_n,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,nutriscore_score,nova_group,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-cerotic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-gondoic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
count,7.959300e+04,7.959300e+04,7.959300e+04,19453.000000,36131.000000,36131.000000,36131.000000,35579.000000,31690.000000,19326.000000,47594.000000,55418.000000,5.000000,54943.000000,51422.000000,1.0,1.0,3.000000,1.0,2.000000,1.0,7.000000,2.000000,1.0,477.000000,475.000000,146.000000,40.000000,3.000000,6.000000e+00,29.000000,5.000000,3.000000,1.0,1.0,1.0,1.0,2.000000,251.000000,241.000000,55129.000000,54726.000000,13.000000,9.000000e+00,1.100000e+01,99.000000,3.000000,1.000,30.00000,113.000000,14055.000000,11.000000,8.000000,54786.000000,49904.000000,49904.000000,2536.000000,377.000000,3.000000,2.710000e+02,449.000000,30.000000,706.000000,400.000000,296.000000,345.000000,440.000000,2.660000e+02,14.000000,4.440000e+02,176.000000,285.000000,5.000000,41.000000,362.000000,110.000000,1217.000000,1.790000e+02,535.000000,599.000000,190.000000,56.000000,54.000000,70.000000,47.000000,2.100000e+01,18.000000,113.000000,46.000000,8.000000,7.000000,346.000000,13.000000,540.000000,1.0,489.000000,22.000000,156.000000,35579.000000,5.0000,3.000000,2.0,5.00000,1.00
mean,2.860990e+35,1.552983e+09,1.592304e+09,134.226385,1.212615,0.011126,0.026404,8.883752,3.210603,1129.487759,266.262468,1095.838304,269.220000,13.580277,5.307997,7.4,6.2,47.433333,18.9,4.051000,3.0,0.005184,0.000665,0.0,23.776689,11.889275,7.297462,5.571745,0.082667,6.894667e-01,15.658966,0.356994,0.000544,2.8,7.3,1.1,5.9,0.678700,0.045135,0.146730,25.421314,12.033902,10.276925,7.321111e+00,1.216455e+01,1.101111,7.366400,0.008,10.17600,59.403363,4.170846,2.900000,2.250000,8.160791,1.362798,0.545087,3.813683,8.138353,0.003327,1.194782e-01,0.321905,0.633593,1.371714,1.429291,0.013529,0.171558,1.071977,8.656025e-01,0.000167,1.529034e-02,0.241619,0.092201,0.118171,0.076468,1.973620,4.117882,1.456739,3.671081e+00,0.213604,0.560881,0.098730,0.008614,0.044992,0.249075,0.297907,1.904876e+00,1.222282,1.880639,1.012103,0.557500,6.214286,40.296590,46.888462,49.003611,15.0,45.944172,130.819290,574.119103,8.883752,0.0524,35.200000,4.0,0.01854,0.03
std,5.266148e+37,5.113993e+07,2.763247e+07,856.584175,1.881728,0.105682,0.190619,8.980991,1.090647,840.502532,667.686511,2604.932654,212.270917,18.219801,8.

In [9]:
#pdfood = pd.read_csv("noT.csv", sep = "\t")

In [10]:
#checking the right position of the columns

concate.filter(col("ingredients_text") != "").select("ingredients_text").show(25)
concate_with_title.filter(col("ingredients_text") != "").select("ingredients_text").show(25)

+--------------------+
|    ingredients_text|
+--------------------+
|Zucker, Kakaomass...|
|Zucker,Palmöl, Ha...|
|Proteinmischung (...|
|Proteinmischung (...|
|100% Soja-Protein...|
|Molkenproteinkonz...|
|Molkenproteinkonz...|
|Molkenproteinkonz...|
|sugar, corn syrup...|
|sugar, corn syrup...|
|Natürliches Miner...|
|Proteinmischung (...|
|100% Soja-Protein...|
|Enthält: 10 mg Me...|
|Tapioca Syrup, Ve...|
|         quinoa 100%|
|Sojaproteinisolat...|
|Heringsfilets (88...|
|Cucumbers, water,...|
|Composition pour ...|
|corn syrup, sugar...|
|water, soybean oi...|
|Branntweinessig, ...|
|Distilled Vinegar...|
|Branntweinessig, ...|
+--------------------+
only showing top 25 rows

+--------------------+
|    ingredients_text|
+--------------------+
|Zucker, Kakaomass...|
|Zucker,Palmöl, Ha...|
|Proteinmischung (...|
|Proteinmischung (...|
|100% Soja-Protein...|
|Molkenproteinkonz...|
|Molkenproteinkonz...|
|Molkenproteinkonz...|
|sugar, corn syrup...|
|sugar, corn syrup...|
|Natürli

- Creator
- Created_datetime
- Last_modified_datetime
- Product_name
- Countries_en
- Traces_en
- Additives_tags
- Main_category_en
- Image_url
- Quantity
- Packaging_tags
- Categories_en
- Ingredients_text
- Additives_en
- Energy-kcal_100g
- Fat_100g
- Saturated-fat_100g
- Sugars_100g
- Salt_100g/sodium_100g

In [11]:
prepared_df = concate.select("creator",
"created_t",                             
"last_modified_t",
"product_name",
"countries_en",
"traces_en",
"additives_en",
"main_category",
"image_url",
"quantity",
"packaging_tags",
"categories",
"ingredients_text",
"additives_n",
"energy-kj_100g",
"fat_100g",
"saturated-fat_100g",
"sugars_100g",
"salt_100g",
"sodium_100g" )

#note: could not find additive, replaced with additive_n

In [12]:
prepared_df.show(3)

+-------------+----------+---------------+--------------------+------------+---------+------------+--------------+--------------------+--------+--------------+-----------+----------------+-----------+--------------+--------+------------------+-----------+---------------+----------------+
|      creator| created_t|last_modified_t|        product_name|countries_en|traces_en|additives_en| main_category|           image_url|quantity|packaging_tags| categories|ingredients_text|additives_n|energy-kj_100g|fat_100g|saturated-fat_100g|sugars_100g|      salt_100g|     sodium_100g|
+-------------+----------+---------------+--------------------+------------+---------+------------+--------------+--------------------+--------+--------------+-----------+----------------+-----------+--------------+--------+------------------+-----------+---------------+----------------+
|      kiliweb|1591989744|     1609478763|    Vitória crackers|     Germany|     null|        null|          null|                nul

In [13]:
prepared_df.filter(col("salt_100g").isNotNull()).count()

49904

# 4. Subset data analysis 

## Find the oldest product:

In [14]:
prepared_df.select("created_t", "product_name").filter(col("product_name").isNotNull()).sort(col("created_t"),ascending = False).show(1,0)

+----------+-------------------+
|created_t |product_name       |
+----------+-------------------+
|1614038706|Vegetarische Wiener|
+----------+-------------------+
only showing top 1 row



## Find the newest product:

In [15]:
prepared_df.select("created_t", "Product_name").sort(col("created_t"),ascending = True).show(1,0)

+----------+----------------+
|created_t |Product_name    |
+----------+----------------+
|1329035567|Milka Ganze Nuss|
+----------+----------------+
only showing top 1 row



## Average product age, where age means how long the product has been in the system:

In [16]:
#convertion from unix format seconds to days to years = 3600*24*365
prepared_df.filter(col("created_t") > 0 ).agg((avg("created_t")/31536000).alias("average time (years)")).show()

+--------------------+
|average time (years)|
+--------------------+
|   49.24477740652864|
+--------------------+



In [17]:
prepared_df.filter(col("created_t") > 0 ).agg((avg("created_t")/31536000).alias("average time (years)")).show()

+--------------------+
|average time (years)|
+--------------------+
|   49.24477740652864|
+--------------------+



## List of other countries where products are sold too:

In [18]:
prepared_df.select(explode(split(col("countries_en"), ",")).alias("indivcountry"))  \
            .filter((col("indivcountry") != "Allemagne") & (col("indivcountry") != "Germany") & (col("indivcountry") != "Deutschland") & ( col("indivcountry") != "en:DE") & ( col("indivcountry") != "en:de")) \
            .select("indivcountry").distinct() \
            .show(15,0) 
   

+----------------------+
|indivcountry          |
+----------------------+
|Middle-east-africa    |
|Côte d'Ivoire         |
|Luxemburgo            |
|Czech-republic-čeština|
|Greece-ελληνικά       |
|Russia                |
|Paraguay              |
|Estados-unidos        |
|Hungary-magyar        |
|Romania-romană        |
|Hong-kong-粵語        |
|Malaysia-中文         |
|Suiza                 |
|Senegal               |
|Tschechien            |
+----------------------+
only showing top 15 rows



In [19]:
from pyspark.sql.functions import countDistinct, avg, stddev

prepared_df.select(explode(split(col("countries_en"), ",")).alias("indivcountry"))  \
            .filter((col("indivcountry") != "Allemagne") & (col("indivcountry") != "Germany") & (col("indivcountry") != "Deutschland") & ( col("indivcountry") != "en:DE") & ( col("indivcountry") != "en:de")) \
            .select(countDistinct("indivcountry").alias("Distinct Countries")).show()

+------------------+
|Distinct Countries|
+------------------+
|               350|
+------------------+



## Identify category of products and compute:
 - Number of products by category
 - List containing names of products by category 

In [20]:
prepared_df.filter(col("Categories") != 'null').groupby("Categories") \
            .agg(count("Product_name").alias("Product_total")) \
            .sort("Product_total",ascending = False) \
            .show(40,0)

+----------------------------------------------------------------------------------------------------------------------------------------+-------------+
|Categories                                                                                                                              |Product_total|
+----------------------------------------------------------------------------------------------------------------------------------------+-------------+
|Milchprodukte, Fermentierte Lebensmittel, Fermentierte Milch, Joghurt                                                                   |394          |
|Milchprodukte, Fermentierte Lebensmittel, Fermentierte Milch, Käse                                                                      |354          |
|Milchprodukte, Fermentierte Lebensmittel, Fermentierte Milch, Käse, Frischkäse                                                          |266          |
|Fleisch, Zubereitetes Fleisch, Würste                                            

In [84]:
#List containing names of products by category
(prepared_df
  .filter(col("Categories") != 'null')
  .groupby("Categories")
  .agg(F.collect_list("Product_name").alias("Product_list"),count("Product_name").alias("product_total"))
  .sort("product_total",ascending =False)
  .show(10))

+--------------------+--------------------+-------------+
|          Categories|        Product_list|product_total|
+--------------------+--------------------+-------------+
|Milchprodukte, Fe...|[LAC Fruchtjoghur...|          394|
|Milchprodukte, Fe...|[Butterkäse cremi...|          354|
|Milchprodukte, Fe...|[Frischkäse Kräut...|          266|
|Fleisch, Zubereit...|[Bierschinken, Pr...|          260|
|Lebensmittel, Saucen|[Plum sauce, Scha...|          240|
|      Brotaufstriche|[Reissirup-Aufstr...|          215|
|Imbiss, Süßwaren,...|[Digestives, Crea...|          207|
|Pflanzliche Leben...|[Gabel Spaghetti,...|          205|
|Imbiss, Süßwaren,...|[Mandel Orange, S...|          198|
|Fleisch, Zubereit...|[Salami geräucher...|          185|
+--------------------+--------------------+-------------+
only showing top 10 rows



## Identify traces and compute: 
 - Number of products by trace 
 - List containing names of products by trace

In [22]:
prepared_df.filter(col("traces_en") != 'null').groupby("traces_en") \
            .agg(count("Product_name").alias("product_total")) \
            .sort("product_total",ascending =False) \
            .show(40,0)

+---------------------------------------+-------------+
|traces_en                              |product_total|
+---------------------------------------+-------------+
|Nuts                                   |1339         |
|Nuts,Peanuts                           |472          |
|Soybeans                               |468          |
|Milk                                   |459          |
|Celery,Mustard                         |451          |
|Sesame seeds                           |326          |
|Celery                                 |249          |
|Gluten,Nuts                            |248          |
|Gluten                                 |239          |
|Milk,Nuts                              |229          |
|Nuts,Soybeans                          |205          |
|Eggs                                   |181          |
|Mustard                                |155          |
|Nuts,Sesame seeds                      |150          |
|Eggs,Gluten,Nuts,Peanuts               |126    

In [27]:
#List containing names of products by trace
(prepared_df
  .filter(col("traces_en") != 'null')
  .groupby("traces_en")
  .agg(F.collect_list("Product_name").alias("Product_list"),count("Product_name").alias("product_total"))
  .sort("product_total",ascending =False)
  .show(10))

+--------------+--------------------+-------------+
|     traces_en|        Product_list|product_total|
+--------------+--------------------+-------------+
|          Nuts|[Baumkuchen, Frui...|         1339|
|  Nuts,Peanuts|[Noisettes grille...|          472|
|      Soybeans|[Hühner-Nudeltopf...|          468|
|          Milk|[Gauda jung, Vita...|          459|
|Celery,Mustard|[Sülzkotelett, Sa...|          451|
|  Sesame seeds|[Flûtes Salées, S...|          326|
|        Celery|[Weißkrautsalat k...|          249|
|   Gluten,Nuts|[Flips Cacahuètes...|          248|
|        Gluten|[Yogo Drink, Lins...|          239|
|     Milk,Nuts|[Frischkäse, Anan...|          229|
+--------------+--------------------+-------------+
only showing top 10 rows



## Data quality analysis on fields of interest (see appendix 1): 
 - Number of products with complete info
 - % of products without complete analysis per 100g
 - % of products without additives info
 - % of products without traces info

In [ ]:
#df filtering relevant columns 

prepared_df.filter((col("Creator").isNotNull()) & 
                   (col("Last_modified_t").isNotNull()) &
                  (col("Product_name").isNotNull()) & 
                   (col("countries_en").isNotNull()) & 
                   (col("traces_en").isNotNull()) &
                   (col("additives_en").isNotNull()) & 
                   (col("Packaging_tags").isNotNull()) & 
                   (col("Categories").isNotNull()) & 
                   (col("Ingredients_text").isNotNull()) & 
                   (col("Energy-kcal_100g").isNotNull()) & 
                   (col("Fat_100g").isNotNull()) &
                   (col("Saturated-fat_100g").isNotNull()) & 
                   (col("Sugars_100g").isNotNull()) & 
                   (col("Salt_100g").isNotNull()) &
                   (col("Sodium_100g").isNotNull())).count()


In [ ]:
#% of products without complete analysis per 100g

productsid = prepared_df.select(col("created_t")).count()
null100 = prepared_df.filter((col("Energy-kcal_100g").isNotNull()) & 
                   (col("Fat_100g").isNotNull()) &
                   (col("Saturated-fat_100g").isNotNull()) & 
                   (col("Sugars_100g").isNotNull()) & 
                   (col("Salt_100g").isNotNull()) &
                   (col("Sodium_100g").isNotNull()) ).count()
(100 * null100 / productsid)

In [ ]:
#% of products without additives info

nulladd = prepared_df.filter( col("additives_en").isNull()).count()
print("Ratio of null additive to all product")
(100 * nulladd / productsid)

In [ ]:
#% of products without traces info

nulltrace = prepared_df.filter( col("traces_en").isNull()).count()
print("Ratio of null traces to all product")
(100 * nulltrace / productsid)

In [ ]:
#dictanalysis = [prepared_df.filter(col(column).isNotNull()).count() for column in prepared_df.schema.names]

## Data profiling on fields of interest (see appendix 1): 
 - Stats on analysis per 100g fields

In [ ]:
print ("Summary of columns Fat_100g, Saturated Sugars_100g, Salt_100g and Sodium_100g:")
prepared_df.select("Fat_100g","Saturated-fat_100g","Sugars_100g","Salt_100g","Sodium_100g").summary().show()

print("Checking for nulls on columns Fat_100g, Saturated Sugars_100g, Salt_100g and Sodium_100g:")
prepared_df.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Fat_100g","Saturated-fat_100g","Sugars_100g","Salt_100g","Sodium_100g"]]).show()

# 5. Data analysis to find out which products are safe:

## Lets check the authorized additives in the EU 

In [31]:
#Authorized additive in the EU 
#Retrieved from https://www.food.gov.uk/business-guidance/approved-additives-and-e-numbers

additives = spark.read \
                 .option("inferSchema", "true") \
                 .option("delimiter", ";") \
                 .option("mode","PERMISSIVE") \
                 .option("header", "true") \
                 .csv("authorised_additive_eu.csv")
additives.show()

+---------+--------------------+
|E numbers|           Additives|
+---------+--------------------+
|     E100|            Curcumin|
|     E101|      (i) Riboflavin|
|     null|(ii) Riboflavin-5...|
|     E102|          Tartrazine|
|     E104|    Quinoline yellow|
|     E110|Sunset Yellow FCF...|
|     E120|Cochineal; Carmin...|
|     E122|Azorubine; Carmoi...|
|     E123|            Amaranth|
|     E124|Ponceau 4R; Cochi...|
|     E127|         Erythrosine|
|     E129|       Allura Red AC|
|     E131|       Patent Blue V|
|     E132|lndigotine; Indig...|
|     E133|  Brilliant Blue FCF|
|     E140|Chlorophylls and ...|
|     E141|Copper complexes ...|
|     E142|             Green S|
|    E150a|       Plain caramel|
|    E150b|Caustic sulphite ...|
+---------+--------------------+
only showing top 20 rows



In [39]:
aditive_df = prepared_df.select(explode(split(col("additives_en"), ",")).alias("additives_list"), "Product_name","Fat_100g","Saturated-fat_100g","Sugars_100g","Salt_100g","Sodium_100g").withColumn("E numbers", regexp_extract("additives_list", r"E\d+",0))
health_df = aditive_df.join(additives,"E numbers",how="left")

health_df.show(3)

+---------+----------------+------------------+--------+------------------+-----------+---------+-----------+------------+
|E numbers|  additives_list|      Product_name|Fat_100g|Saturated-fat_100g|Sugars_100g|Salt_100g|Sodium_100g|   Additives|
+---------+----------------+------------------+--------+------------------+-----------+---------+-----------+------------+
|     E322|E322 - Lecithins|Belgische Pralinen|    33.3|              21.1|       51.5|     0.09|      0.036|   Lecithins|
|     E420| E420 - Sorbitol|Belgische Pralinen|    33.3|              21.1|       51.5|     0.09|      0.036|(i) Sorbitol|
|     E422| E422 - Glycerol|Belgische Pralinen|    33.3|              21.1|       51.5|     0.09|      0.036|    Glycerol|
+---------+----------------+------------------+--------+------------------+-----------+---------+-----------+------------+
only showing top 3 rows



In [98]:
#Potential danger products?

non_registeredA = health_df.filter(col("Additives").isNull()).select("Product_name","additives_list")
non_registeredA.show(10,0)
print("total products with unkown additives ")
non_registeredA.count()

+--------------------------------+---------------------------------------------------------------------------------------+
|Product_name                    |additives_list                                                                         |
+--------------------------------+---------------------------------------------------------------------------------------+
|Sour Fruit Gummies              |E428 - Gelatine                                                                        |
|Vita Cola pur                   |E150d - Sulphite ammonia caramel                                                       |
|Chocolate and caramel candy     |E1400 - Dextrin                                                                        |
|Zesty Italian                   |E160b - Annatto                                                                        |
|Bröd Mjukkaka                   |E472e - Mono- and diacetyltartaric acid esters of mono- and diglycerides of fatty acids|
|Sweet Baby Ray'

5299

## Lets add food lights for the health factor of each product 

In [79]:
health_dflight = health_df.withColumn('foodlighting', when(col("Additives").isNull(), "darkred")\
                                                     .when((col("Sugars_100g") > 22.5) | (col("Fat_100g") > 17.5) | (col("Salt_100g") > 1.5) | (col("Saturated-fat_100g") > 5), "red")\
                                                    .otherwise(when((col("Sugars_100g") > 5) | (col("Fat_100g") > 3) | (col("Salt_100g") > 0.3) | (col("Saturated-fat_100g") > 1.5), "orange").otherwise("green"))\
                                                    )\
                        .select("Product_name","foodlighting").distinct()

In [80]:
health_dflight.show(10,0)

+---------------------------------+------------+
|Product_name                     |foodlighting|
+---------------------------------+------------+
|Gaufrettes fines                 |red         |
|French Dressing                  |red         |
|Chorizo-salami - Dulano - 125 G  |red         |
|Double Concentré de Tomates      |orange      |
|Crunchy muesli - Chocolate & Nuts|red         |
|Pralinés Selection               |darkred     |
|Hähnchenbrustfilet, Klassik      |red         |
|Ce'Real                          |red         |
|Kabanossin mild                  |red         |
|Curry gewürz ketchup             |red         |
+---------------------------------+------------+
only showing top 10 rows



In [85]:
health_dflight.filter(col("foodlighting") == 'green').show(10,0)
health_dflight.filter(col("foodlighting") == 'orange').show(10,0)
health_dflight.filter(col("foodlighting") == 'red').show(10,0)

+--------------------------------------+------------+
|Product_name                          |foodlighting|
+--------------------------------------+------------+
|High protein chocolate pudding        |green       |
|Extra Professionell White             |green       |
|Guma rozpuszczalna o smakach owocowych|green       |
|Thymian Lutsch Pastillen              |green       |
|Apfel-Grape                           |green       |
|Heilwasser mit Kohlensäure            |green       |
|Rocka Milk                            |green       |
|Xucker Light                          |green       |
|Fein gehackte Tomaten in Tomatensaft  |green       |
|Boemboe Nasi Goreng                   |green       |
+--------------------------------------+------------+
only showing top 10 rows

+-----------------------------------+------------+
|Product_name                       |foodlighting|
+-----------------------------------+------------+
|Double Concentré de Tomates        |orange      |
|Schweppes Rus

## What is the ratio for each lights given our dataset

In [82]:
healthy = health_dflight.filter(col("foodlighting") == 'green').count()
midhealthy = health_dflight.filter(col("foodlighting") == 'orange').count()
unhealthy = health_dflight.filter(col("foodlighting") == 'red').count()
unauthorized = health_dflight.filter(col("foodlighting") == 'darkred').count()
allP = health_dflight.count()
#Healthy food ratio
print("Healthy products: "+ str((healthy/allP)*100))
#Unhealthy product
print("Unhealthy products: "+ str((unhealthy/allP)*100))
#midhealthy product 
print("Midhealthy products: "+ str((midhealthy/allP)*100))
#forbidden additives 
print("Forbidden products: "+ str((unauthorized/allP)*100))

Healthy products: 10.17864099282097
Unhealthy products: 45.56736601925538
Midhealthy products: 23.050809727864653
Forbidden products: 21.203183260058992


## Lets apply our analysis to one of the most famous German product: Wurst  (sausages) :)

In [93]:
df = health_dflight.withColumn('sausages',when((col('Product_name').contains('Wurst')) | (col('Product_name').contains('wurst'))| (col('Product_name').contains('sausages')),True).otherwise(False))

In [94]:
df.filter(col('sausages') == True).select(col("*")).distinct().count()

284

In [95]:
df.filter(col('sausages') == True).select(col("*")).distinct().show(10,0)

+------------------------------------+------------+--------+
|Product_name                        |foodlighting|sausages|
+------------------------------------+------------+--------+
|Biewurst                            |red         |true    |
|Gutsmettwurst                       |red         |true    |
|Oma Hildes Leberwurst Hausmacher Art|red         |true    |
|Geflügel-Bratwurst                  |red         |true    |
|Original Pfälzer Bratwurst          |red         |true    |
|Kaiser-Jagdwurst                    |red         |true    |
|Rostbratwurst / Schwein/Grill       |red         |true    |
|Sahne-Leberwurst                    |red         |true    |
|Schinken-Zwiebelmettwurst           |red         |true    |
|Fleischwurst geräuchert             |red         |true    |
+------------------------------------+------------+--------+
only showing top 10 rows

